In [1]:
import argparse
import itertools
import pathlib

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import toml
from joblib import dump
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

In [2]:
argparser = argparse.ArgumentParser()
argparser.add_argument("--cell_type", default="all")

args = argparser.parse_args()

cell_type = args.cell_type

In [3]:
# Parameters
aggregation = True
nomic = True

In [4]:
MODEL_TYPE = "regression"

In [5]:
# toml file path
TOML_PATH = pathlib.Path("../splits.toml")
# read toml file via toml
data_splits_by_treatments = toml.load(TOML_PATH)

# define the 100% test set data treatments
test_100_percent = data_splits_by_treatments["splits"]["data_splits_100"]
test_75_percent = data_splits_by_treatments["splits"]["data_splits_75"]

In [6]:
path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
)

data_df = pd.read_parquet(path)

data_df.head()

,Metadata_Well,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,...,uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],oneb_Treatment_Dose_Inhibitor_Dose
0,B02,0.100173,-0.059734,0.218567,0.111938,0.007420,-0.100946,-0.030356,-0.070701,0.013108,...,0.396902,0.385081,1.000000,0.000000,0.430111,0.538503,0.784695,0.468448,0.237545,LPS_0.010_ug_per_ml_DMSO_0.025_%
1,B03,0.137279,-0.097646,0.205644,0.108021,-0.002159,-0.141895,-0.059932,-0.091195,-0.011037,...,0.256691,0.327491,0.390866,0.406489,0.412096,0.104830,0.812933,0.518536,0.244397,LPS_0.010_ug_per_ml_DMSO_0.025_%
2,B04,0.071345,-0.053566,0.055404,0.013373,0.004443,-0.111708,-0.084402,-0.043409,-0.030164,...,0.555221,0.357476,0.346884,0.477553,0.427658,0.642061,0.249380,0.627712,0.318350,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...
3,B05,0.110685,-0.084346,0.107954,0.071923,0.004150,-0.121376,-0.075382,-0.052805,-0.038156,...,0.308536,0.588899,0.828371,0.484102,0.294634,0.673648,0.236793,0.557634,0.350429,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...
4,B06,-0.021771,0.018442,-0.048689,-0.070490,-0.005284,-0.008255,-0.012815,-0.017174,0.003785,...,0.469875,0.395392,0.560129,0.504521,0.490444,0.258834,0.238358,0.524276,0.250670,DMSO_0.100_%_DMSO_0.025_%


In [7]:
# subset each column that contains metadata
metadata = data_df.filter(regex="Metadata")

# get all columns that are not metadata except for metadata_Well
data = data_df.drop(metadata.columns, axis=1)

# get the metadata_Well column
metadata_well = metadata[
    ["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
]

data_df = pd.merge(data, metadata_well, left_index=True, right_index=True)

This model and code is both inspired and reused from: https://github.com/WayScience/phenotypic_profiling_model/blob/main/1.split_data/split_data.ipynb
The bulk of this work was done by **Roshan Kern** I have only made minor changes to the code to make it more modular and easier to use for my purposes.

In [8]:
# variable test and train set splits
# 100% test set
# subset the following treatments for test set
test_set_all = data_df[
    data_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(test_100_percent)
]
# 75% test set and 25% train set
test_set_75 = data_df[
    data_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(test_75_percent)
]

test_100_and_75 = test_100_percent + test_75_percent

# 50% test set and 50% train set
# get all treatments that are not in the_test_set_all and the test_set_75
test_set_50 = data_df[
    ~data_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(test_100_and_75)
]

print(test_set_all.shape, test_set_75.shape, test_set_50.shape)

(8, 1389) (8, 1389) (138, 1389)


In [9]:
# get the train test splits from each group

# 75% test set and 25% train set
test_ratio = 0.75
training_data_set_75, testing_data_set_75 = train_test_split(
    test_set_75,
    test_size=test_ratio,
    stratify=test_set_75["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
    random_state=0,
)

# 50% test set and 50% train set
test_ratio = 0.5
training_data_set_50, testing_data_set_50 = train_test_split(
    test_set_50,
    test_size=test_ratio,
    stratify=test_set_50["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
    random_state=0,
)

# verify that the correct splits have been made
# 100% test set
print(f"Shape for the 100% test set: {test_set_all.shape}\n")

# 75% test set and 25% train set
print(
    f"Shape for the 75% test set: {training_data_set_75.shape};\nShape for the 75% train set: {testing_data_set_75.shape}\n"
)

# 50% test set and 50% train set
print(
    f"Shape for the 50% test set: {training_data_set_50.shape};\nShape for the 50% train set: {testing_data_set_50.shape}"
)

Shape for the 100% test set: (8, 1389)

Shape for the 75% test set: (2, 1389);
Shape for the 75% train set: (6, 1389)

Shape for the 50% test set: (69, 1389);
Shape for the 50% train set: (69, 1389)


In [10]:
# combine all testing sets together while preserving the index
testing_data_set = pd.concat(
    [test_set_all, testing_data_set_75, testing_data_set_50], axis=0
)
testing_data_set = testing_data_set.sort_index()

# combine all training sets together while preserving the index
training_data_set = pd.concat([training_data_set_75, training_data_set_50], axis=0)
training_data_set = training_data_set.sort_index()

print(
    f"Testing set length: {len(testing_data_set)}\nTraining set length: {len(training_data_set)}"
)

# get the indexes for the training and testing sets
testing_data_set_index = testing_data_set.index
training_data_set_index = training_data_set.index

Testing set length: 83
Training set length: 71


In [11]:
# create pandas dataframe with all indexes and their respective labels, stratified by phenotypic class
index_data = []
for index in training_data_set_index:
    index_data.append({"labeled_data_index": index, "label": "train"})
for index in testing_data_set_index:
    index_data.append({"labeled_data_index": index, "label": "test"})

# make index data a dataframe and sort it by labeled data index
index_data = pd.DataFrame(index_data).sort_values(["labeled_data_index"])

In [12]:
# set the save path
save_path = pathlib.Path(f"../indexes/{cell_type}/{MODEL_TYPE}")
print(save_path)
# create save path if it doesn't exist
save_path.mkdir(parents=True, exist_ok=True)

../indexes/PBMC/regression


In [13]:
# save indexes as tsv file
if aggregation == True:
    if nomic == True:
        index_data.to_csv(
            f"{save_path}/aggregated_sc_and_nomic_data_split_indexes.tsv", sep="\t"
        )
    elif nomic == False:
        index_data.to_csv(f"{save_path}/aggregated_sc_data_split_indexes.tsv", sep="\t")
elif aggregation == False:
    if nomic == True:
        index_data.to_csv(f"{save_path}/sc_and_nomic_data_split_indexes.tsv", sep="\t")
    elif nomic == False:
        index_data.to_csv(f"{save_path}/sc_split_indexes.tsv", sep="\t")
else:
    print("Error: Check aggregation and nomic parameters as they are not set correctly")